In [8]:
import pandas as pd
import numpy as np
df = pd.read_csv('shuffle.csv')
df.head()

,team1_seed,team2_seed,team1_dis(km),team2_dis(km),team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,team1_pt_overall_ff,...,team2_stlrate,team2_oppstlrate,team1_adjtempo,team1_adjoe,team1_adjde,team2_adjtempo,team2_adjoe,team2_adjde,team1_outcome,game_id
0,16,16,1076.682624,932.116689,6,6,0,0,0,0,...,0.0778,0.1020,73.8504,99.8665,106.4500,67.7359,98.6102,99.6885,0,2002-1373-1108
1,2,15,969.775258,1058.567916,0,2,0,0,0,0,...,0.1282,0.1076,69.9001,111.4954,93.8770,71.2446,96.8669,99.9263,1,2002-1104-1194
2,14,3,1210.852647,513.529264,0,0,0,0,0,0,...,0.0835,0.0839,64.7948,105.4534,97.6704,72.8207,117.3877,96.9262,0,2002-1112-1364
3,16,1,339.990955,148.384590,3,3,0,0,0,0,...,0.1297,0.0919,67.4639,95.3632,99.9754,75.1685,118.5999,87.7504,0,2002-1181-1457
4,5,12,3007.304604,859.001991,1,1,0,0,0,0,...,0.0922,0.0927,66.5739,109.6136,89.9224,63.3547,108.6231,95.6459,1,2002-1231-1428


In [9]:
game_id = df['game_id'].tolist()
df = df.drop(['game_id'],axis = 1)
df.head()

,team1_seed,team2_seed,team1_dis(km),team2_dis(km),team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,team1_pt_overall_ff,...,team2_opparate,team2_stlrate,team2_oppstlrate,team1_adjtempo,team1_adjoe,team1_adjde,team2_adjtempo,team2_adjoe,team2_adjde,team1_outcome
0,16,16,1076.682624,932.116689,6,6,0,0,0,0,...,57.8947,0.0778,0.1020,73.8504,99.8665,106.4500,67.7359,98.6102,99.6885,0
1,2,15,969.775258,1058.567916,0,2,0,0,0,0,...,56.3399,0.1282,0.1076,69.9001,111.4954,93.8770,71.2446,96.8669,99.9263,1
2,14,3,1210.852647,513.529264,0,0,0,0,0,0,...,54.8255,0.0835,0.0839,64.7948,105.4534,97.6704,72.8207,117.3877,96.9262,0
3,16,1,339.990955,148.384590,3,3,0,0,0,0,...,49.5516,0.1297,0.0919,67.4639,95.3632,99.9754,75.1685,118.5999,87.7504,0
4,5,12,3007.304604,859.001991,1,1,0,0,0,0,...,51.5702,0.0922,0.0927,66.5739,109.6136,89.9224,63.3547,108.6231,95.6459,1


In [10]:
array = df.values
x = array[:,0:66]
y = array[:,66]
x.shape, y.shape

((1112, 66), (1112,))

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import cross_validation
sc = StandardScaler()
x_std = sc.fit_transform(x)
pca = PCA(n_components = 'mle',svd_solver='full')
x_pca = pca.fit_transform(x_std)
x_train_pca, x_test_pca, y_train, y_test = cross_validation.train_test_split(x_pca,y,test_size=0.3,random_state = 42)
x_train_pca.shape,x_test_pca.shape

((778, 65), (334, 65))

In [12]:
# logistic
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr = LogisticRegression(random_state=1)
param_grid = [{'penalty':['l1','l2'],
                   'C':[0.01,0.05,0.1,0.5,1,5,10,50,100],
                    'solver':['liblinear'],
                    'multi_class':['ovr']},
                {'penalty':['l2'],
                 'C':[0.01,0.05,0.1,0.5,1,5,10,50,100],
                'solver':['lbfgs'],
                'multi_class':['ovr','multinomial']}]
search = GridSearchCV(lr, param_grid, cv=10,scoring='accuracy')
search.fit(x_train_pca, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.699):
{'C': 0.01, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'liblinear'}


In [6]:
# 返回probability 第二列表示该场比赛team1 win的概率
y_prob = search.predict_proba(x_pca)
from sklearn.metrics import accuracy_score
y_pred = search.predict(x_pca)
accuracy = accuracy_score(y, y_pred)
accuracy

0.7221223021582733

In [19]:
df['team1_outcome_pred'] = y_pred[:]

In [20]:
team1_win = []
for lst in y_prob:
    p = lst[1]
    team1_win.append(p)
team1_win

[0.467505165490484,
 0.8462127921722551,
 0.1601288900859359,
 0.10620295603256777,
 0.5323200577044933,
 0.09861328129507183,
 0.5513064017802142,
 0.6906557235902134,
 0.3216784457535987,
 0.5333483066084528,
 0.7954199878153525,
 0.8526522014632988,
 0.5567395381886903,
 0.5927533984222131,
 0.6840728261714744,
 0.49044188891099305,
 0.6644702762477844,
 0.35614584488944623,
 0.08000738881630594,
 0.19094434873104557,
 0.7218337744593285,
 0.21964629948459913,
 0.7575002542137669,
 0.10088749784388086,
 0.678761199570456,
 0.42504586079858353,
 0.11161569727992976,
 0.7908313199674125,
 0.664409924804794,
 0.6625050503618036,
 0.6077703828224768,
 0.5091138948384769,
 0.22281690420124453,
 0.7908721425197407,
 0.8386588350929493,
 0.2724749544926348,
 0.16100993105595265,
 0.5748019041662569,
 0.36191056323093734,
 0.3150768302058005,
 0.2783409348594532,
 0.6173509144184082,
 0.22230546918220476,
 0.7440223704236169,
 0.27899628014933414,
 0.3080706445448881,
 0.28885772473413046,


In [21]:
df['game_id'] = game_id
df['probability'] = team1_win
df.head()

,team1_seed,team2_seed,team1_dis(km),team2_dis(km),team1_pt_school_ncaa,team1_pt_overall_ncaa,team1_pt_school_s16,team1_pt_overall_s16,team1_pt_school_ff,team1_pt_overall_ff,...,team1_adjtempo,team1_adjoe,team1_adjde,team2_adjtempo,team2_adjoe,team2_adjde,team1_outcome,team1_outcome_pred,game_id,probability
0,16,16,1076.682624,932.116689,6,6,0,0,0,0,...,73.8504,99.8665,106.4500,67.7359,98.6102,99.6885,0,0.0,2002-1373-1108,0.467505
1,2,15,969.775258,1058.567916,0,2,0,0,0,0,...,69.9001,111.4954,93.8770,71.2446,96.8669,99.9263,1,1.0,2002-1104-1194,0.846213
2,14,3,1210.852647,513.529264,0,0,0,0,0,0,...,64.7948,105.4534,97.6704,72.8207,117.3877,96.9262,0,0.0,2002-1112-1364,0.160129
3,16,1,339.990955,148.384590,3,3,0,0,0,0,...,67.4639,95.3632,99.9754,75.1685,118.5999,87.7504,0,0.0,2002-1181-1457,0.106203
4,5,12,3007.304604,859.001991,1,1,0,0,0,0,...,66.5739,109.6136,89.9224,63.3547,108.6231,95.6459,1,1.0,2002-1231-1428,0.532320


In [22]:
outcome = df.drop(df.columns[0:68],axis = 1)

In [23]:
outcome

,game_id,probability
0,2002-1373-1108,0.467505
1,2002-1104-1194,0.846213
2,2002-1112-1364,0.160129
3,2002-1181-1457,0.106203
4,2002-1231-1428,0.532320
5,2002-1242-1221,0.098613
6,2002-1245-1329,0.551306
7,2002-1246-1434,0.690656
8,2002-1281-1274,0.321678
9,2002-1323-1150,0.533348


In [7]:
# outcome.to_csv('result_logistic.csv')
from sklearn.metrics import log_loss
log_loss(y, y_prob)

0.5424264479412801

In [13]:
#model = tree.DecisionTreeClassifier()
#model = tree.DecisionTreeRegressor()
#model = linear_model.LogisticRegression()
#model = linear_model.BayesianRidge()
#model = linear_model.Lasso()
#model = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.0001, C=0.1)
#model = svm.SVC()
#model = svm.SVR()
#model = linear_model.Ridge(alpha = 0.5)
#model = neuralNetwork(10)
#model = AdaBoostClassifier(n_estimators=100)
#model = RandomForestClassifier(n_estimators=64)
#model = KNeighborsClassifier(n_neighbors=39)
#model = VotingClassifier(estimators=[('GBR', model1), ('BR', model2), ('KNN', model3)], voting='soft')
#model = GradientBoostingClassifier(n_estimators=100)
#model = GradientBoostingRegressor(n_estimators=100, max_depth=5)

In [3]:
import pandas as pd
df_seed = pd.read_csv('spss_pred.csv')
df_seed.head()

,team1_outcome,$LP-0,$LP-1
0,0,0.653,0.347
1,1,0.022,0.978
2,0,0.959,0.041
3,0,0.992,0.008
4,1,0.517,0.483


In [5]:
from sklearn.metrics import log_loss
array = df_seed.values
x = array[:,0:1]
y = array[:,1:3]
log_loss(x, y)

0.5071232361606842